In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits

#from tiger_cutout_hsc import cutout_one, prepare_catalog # generate_cutout, get_psf
#import lsst.daf.persistence as dafPersist

In [3]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [25]:
# Select 100 galaxies
lsbg_cat = Table.read('./Catalog/lsbg_NSA_z005.fits') # This contains both candy+galaxy and junks

rnd = np.random.randint(0, len(lsbg_cat), 50)

# rnd = np.array([5639, 2575,  782,  865,  636, 1205, 3672, 6502, 2581,  531, 6369,
#        5619, 5341, 1798,  472,  742,  523, 5345, 6639, 5890, 4942, 4686,
#        2807,  161,  565, 5138, 3618,   95, 3581, 4915, 4825, 6124, 4654,
#        3804, 1685,  583, 4148, 3393, 7455, 1085, 4428, 5972, 2193,  150,
#        6056, 1109, 1008, 2546, 6907, 4958])
cat = lsbg_cat[rnd]

### Adaptive cutout size

In [26]:
## Adaptive cutout size??  
# Normal objects, use 1.0 arcmin cutout.
# Radius > 20 arcsec, use 1.5 arcmin cutout.
# Radius > 30 arcsec, use 2.0 arcmin cutout.
size = np.ones(len(cat)) * 1.0 * u.arcmin

size[cat['flux_radius_ave_i'] > 20] = 1.5 * u.arcmin # shoud be larger
size[cat['flux_radius_ave_i'] > 30] = 2 * u.arcmin # should be larger 

cat['cutout_size'] = size.value
cat.write('./Catalog/nsa_test_sample_210927.fits', overwrite=True)

### Generate cutout

Run `'/home/jiaxuanl/Research/Packages/kuaizi/diezi/gen_cutout/nsa_sample/lsbg_cutout_nsa.sh'`

In [ ]:
#### Download quick RGB from Johnny's website
import urllib

temp = cat
for k, idx in enumerate(temp['viz-id'].data):
    suffix = ''
    if temp[k]['is_galaxy'] >= 1:
        suffix += '_gal'
    if temp[k]['is_candy'] >= 1:
        suffix += '_cd'
    if temp[k]['bad_votes'] >= 1:
        suffix += '_jk'
        
    url = f'https://hugs-detections.nyc3.digitaloceanspaces.com/hugs-{idx}.png'
    urllib.request.urlretrieve(url, filename=f'{idx}' + suffix + '.png')